In [1]:
import requests
import pandas as pd
import time
import random
import json
import os
from dotenv import load_dotenv

load_dotenv()

cookie_value = os.getenv("COOKIE")

# ==========================================
# 1. การตั้งค่า (Configuration)
# ==========================================
START_PAGE = 1
END_PAGE = 13
LIMIT = 20
OUTPUT_FILENAME = "cedt_intern_data_paginated" + str(END_PAGE) + ".csv"

# URL พร้อม Query Parameters (เว้น page ไว้ใส่ค่า)
API_URL_TEMPLATE = "https://cedtintern.cp.eng.chula.ac.th/api/sessions/5/openings?search=&page={}&limit={}&onlyBookmarked=false&onlyAvailablePositions=false"

# *** ใส่ Cookie ของคุณที่นี่ ***
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Cookie": cookie_value
}

all_job_data = []

# ==========================================
# 2. เริ่มการวนลูปทีละหน้า (Pagination Loop)
# ==========================================
print(f"Starting scraping pages {START_PAGE} to {END_PAGE}...")

for page in range(START_PAGE, END_PAGE + 1):
    # สร้าง URL โดยใส่เลขหน้าและ limit
    url = API_URL_TEMPLATE.format(page, LIMIT)
    
    try:
        print(f"Fetching Page {page}...", end=" ")
        response = requests.get(url, headers=HEADERS, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            items = data.get("items", [])
            
            print(f"Found {len(items)} items.")
            
            # วนลูปดึงข้อมูลย่อยในแต่ละ Page (Iterate items in page)
            for item in items:
                # Mapping ข้อมูลให้ชื่อ Column ตรงกับไฟล์ CSV เก่า
                job_info = {
                    "id": item.get("openingId"),
                    "company_name": item.get("company", {}).get("companyNameTh"),
                    "position_title": item.get("title"),
                    "quota": item.get("quota"),
                    "salary_amount": item.get("compensationAmount"),
                    # จัดการกรณี salary_type อาจเป็น None
                    "salary_type": item.get("compensationType", {}).get("compensationType") if item.get("compensationType") else None,
                    "work_type": item.get("workingCondition"),
                    "location": item.get("officeName"),
                    # รวม Tags
                    "tags": ", ".join([t['tagName'] for t in item.get("tags", [])]),
                    "description_html": item.get("description"),
                    "api_url": url  # เก็บ URL หน้า list ไว้เป็น reference
                }
                
                all_job_data.append(job_info)
                
        else:
            print(f"[ERR] Status {response.status_code}")

    except Exception as e:
        print(f"[ERR] Exception: {e}")

    # ==========================================
    # 3. หน่วงเวลาแบบสุ่ม (Random Delay)
    # ==========================================
    # แม้จะยิงน้อยครั้ง แต่ควรหน่วงเวลาเล็กน้อยเพื่อความปลอดภัย
    delay = random.uniform(2.0, 5.0)
    time.sleep(delay)

# ==========================================
# 4. บันทึกผลลัพธ์ (Export)
# ==========================================
print("-" * 30)
if all_job_data:
    df = pd.DataFrame(all_job_data)
    
    # บันทึกไฟล์
    df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
    
    print(f"Scraping Finished! Total jobs collected: {len(df)}")
    print(f"Saved to: {OUTPUT_FILENAME}")
    print(df.head())
else:
    print("No data found. Please check Cookie.")

Starting scraping pages 1 to 13...
Fetching Page 1... Found 20 items.
Fetching Page 2... Found 20 items.
Fetching Page 3... Found 20 items.
Fetching Page 4... Found 20 items.
Fetching Page 5... Found 20 items.
Fetching Page 6... Found 20 items.
Fetching Page 7... Found 20 items.
Fetching Page 8... Found 20 items.
Fetching Page 9... Found 20 items.
Fetching Page 10... Found 20 items.
Fetching Page 11... Found 5 items.
Fetching Page 12... Found 5 items.
Fetching Page 13... Found 5 items.
------------------------------
Scraping Finished! Total jobs collected: 215
Saved to: cedt_intern_data_paginated13.csv
     id                                       company_name  \
0  1039              บริษัท บี.กริม เพาเวอร์ จำกัด (มหาชน)   
1  1043                               บริษัท อโยเดีย จำกัด   
2  1046                      บริษัท ดีมีเตอร์ ไอซีที จำกัด   
3  1055  บริษัท ช็อคโก้ คาร์ด เอ็นเตอร์ไพรส์ จำกัด (Cho...   
4  1061                  ธนาคารกรุงศรีอยุธยา จำกัด (มหาชน)   

                 